In [ ]:
# IMPORTAR LIBRERIAS NECESARIAS

import os #trabajar con el sistema operativo y variables de entorno
from dotenv import load_dotenv #cargar variable de entorno
import spotipy #librería para trabajar con la API de Spotify
from spotipy.oauth2 import SpotifyClientCredentials #autenticación con Spotify
import pandas as pd #dataframes
from urllib.parse import quote #esta librería sirve para codificar las URLs (espacios y caracteres especiales -> %20, etc)
import numpy as np
import mysql.connector
from mysql.connector import Error

In [3]:
load_dotenv() #carga las variables del entorno .env; devuelve un true o false

#Spotify
mis_credenciales = SpotifyClientCredentials(
    client_id=os.getenv("SPOTIFY_CLIENT_ID"),
    client_secret=os.getenv("SPOTIFY_CLIENT_SECRET")
)
spotify = spotipy.Spotify(auth_manager=mis_credenciales)

#last fm
api_key_lastfm = os.getenv("API_KEY_LASTFM")
shared_secret_lastfm = os.getenv("SHARED_SECRET_LASTFM")

#MySQL
MYSQL_HOST = os.getenv("MYSQL_HOST")
MYSQL_USER = os.getenv("MYSQL_USER")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")

In [4]:
try:
    cnx = mysql.connector.connect(
        host= MYSQL_HOST,
        user= MYSQL_USER,
        password= MYSQL_PASSWORD,
        database="musicstream_db"
    )
    print('Conexión exitosa')
except Error as e:
    print('Error al conectar:', e)



Conexión exitosa


In [7]:
## Identificar artistas y géneros más populares.
#Incio del  cursor
mycursor = cnx.cursor()

 #Mostrar resultados directamente desde pandas con el metodo pd.read_sql_query
query1 = """SELECT artista, genero, SUM(OYENTES) AS total_oyentes
FROM canciones  \
GROUP BY artista,genero \
ORDER BY total_oyentes DESC \
LIMIT 10;"""

top_10_artistas = pd.read_sql_query(query1,cnx)
top_10_artistas

C:\Users\Usuario\AppData\Local\Temp\ipykernel_22520\2226554288.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  top_10_artistas = pd.read_sql_query(query1,cnx)


,artista,genero,total_oyentes
0,Queen,rock,1.531373e+10
1,The Beatles,rock,1.262024e+10
2,Muse,rock,1.250241e+10
3,R.E.M.,rock,1.070761e+10
4,Johnny Cash,country,1.053405e+10
5,U2,rock,6.673696e+09
6,Louis Armstrong,jazz,5.861870e+09
7,Ella Fitzgerald,jazz,4.270203e+09
8,Elvis Presley,rock,4.223814e+09
9,Nirvana,rock,3.512138e+09


In [8]:
# Años con más albumes.
mycursor = cnx.cursor()

query2 = """SELECT lanzamiento AS año,COUNT(DISTINCT album) AS total_albumes
FROM canciones
GROUP BY lanzamiento
ORDER BY total_albumes DESC;"""

años_mas_albumes = pd.read_sql_query(query2,cnx)
años_mas_albumes

C:\Users\Usuario\AppData\Local\Temp\ipykernel_22520\3579415350.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  años_mas_albumes = pd.read_sql_query(query2,cnx)


,año,total_albumes
0,2018,1411
1,2014,1255
2,2016,1134
3,2012,1061
4,2010,870


In [9]:
# Años con más canciones
mycursor = cnx.cursor()

query = """SELECT lanzamiento AS año,COUNT(*) AS total_canciones
FROM canciones
GROUP BY lanzamiento
ORDER BY total_canciones DESC;"""

años_mas_canciones = pd.read_sql_query(query,cnx)
años_mas_canciones


C:\Users\Usuario\AppData\Local\Temp\ipykernel_22520\3601634418.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  años_mas_canciones = pd.read_sql_query(query,cnx)


,año,total_canciones
0,2014,13980
1,2012,13498
2,2010,10797
3,2018,10166
4,2016,9290


In [ ]:
# ¿Qué género tuvo más lanzamientos en 2014?

mycursor = cnx.cursor()

query = """SELECT genero,COUNT(*) AS total_canciones
FROM canciones
WHERE lanzamiento = 2014
GROUP BY GENERO
ORDER BY total_canciones DESC
LIMIT 1;"""

genero_mas_canciones = pd.read_sql_query(query,cnx)
genero_mas_canciones


C:\Users\Usuario\AppData\Local\Temp\ipykernel_22520\666810639.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  genero_mas_canciones = pd.read_sql_query(query,cnx)


,genero,total_canciones
0,jazz,5715


In [11]:
# Top 3 de artistas con más lanzamientos en 2014.

mycursor = cnx.cursor()

query = """SELECT artista,COUNT(*) AS total_canciones
FROM canciones
WHERE lanzamiento = 2014
GROUP BY artista
ORDER BY total_canciones DESC
LIMIT 3;"""

top_3_artistas_2014 = pd.read_sql_query(query,cnx)
top_3_artistas_2014


C:\Users\Usuario\AppData\Local\Temp\ipykernel_22520\4186486353.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  top_3_artistas_2014 = pd.read_sql_query(query,cnx)


,artista,total_canciones
0,joão gilberto,1375
1,arcángel,803
2,louis armstrong,652


In [12]:
# Año con menor producción musical

mycursor = cnx.cursor()

query = """SELECT lanzamiento AS año, COUNT(*) AS total_canciones
FROM canciones
GROUP BY lanzamiento
ORDER BY total_canciones ASC
LIMIT 1;
;"""

año_menos_lanzamientos = pd.read_sql_query(query,cnx)
año_menos_lanzamientos


C:\Users\Usuario\AppData\Local\Temp\ipykernel_22520\897658712.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  año_menos_lanzamientos = pd.read_sql_query(query,cnx)


,año,total_canciones
0,2016,9290


In [13]:
## Cerrar la conexion con MySQL

cnx.close()